In [ ]:
import pandas as pd
import json
import datetime

# Hilfsfunktion für das Datum in den JSON Statements

In [ ]:
# von https://stackoverflow.com/questions/46906432/how-to-parse-datetime-that-ends-with-z
def string_to_datetime(date_string):
    date_format = '%Y-%m-%dT%H:%M:%S'
    date = datetime.datetime.strptime(date_string.split('.')[0], date_format)
    date = date + datetime.timedelta(microseconds = int(date_string.split('.')[1][:-1])/1000)
    return date

def datetime_to_string(date):
    date_format = '%Y-%m-%dT%H:%M:%S'
    date_string = date.strftime(date_format) + '.000Z'
    return date_string

# JSON Datei lesen und verändern

In [ ]:
with open('moodle-dashboard.json') as file:
    data = json.load(file)
    
updated_data = data
    
# jede Beantwortung einer der datensouveränen Fragen heraussuchen
temp_data = []  
i = 0

while i < len(data):
    if 'Auswahl der Ã¼bermittelten Daten'  in str(data[i]['context']['contextActivities']):
        if 'answered' in str(data[i]['verb']['display']):
            temp_data.append(data[i])
    i += 1

In [ ]:
# DataFrame mit allen wichtigen Informationen erstellen
df_data = []

i = 0
while i < len(temp_data):
    language = temp_data[i]['context']['language']
    df_data.append([temp_data[i]['actor']['name'],
                    temp_data[i]['actor']['mbox'],
                    temp_data[i]['object']['definition']['name'][language], 
                    temp_data[i]['result']['response'],
                    temp_data[i]['timestamp']
                   ])
    i += 1

df = pd.DataFrame(df_data, columns = ['Name', 'E-Mail', 'Frage', 'Antwort', 'Zeitpunkt'])

name_list = df['Name'].unique().tolist()
question_list = df['Frage'].unique().tolist()
question_list[0], question_list[6] = question_list[6], question_list[0]

In [ ]:
# es könnte nicht notwendig sein ein komplettes statements zu löschen: 
# die Informationen können ohne private Informationen genutzt werden
# diese Funktion wurde in dieser Arbeit nicht genutzt 
def delete_statement(statement_ids):
    new_statements = []
    i = 0
    while i < len(updated_data):
        for statement_id in statement_ids:
            if updated_data[i]['id'] != statement_id:
                new_statements.append(updated_data[i])
        i += 1
    return new_statements

In [ ]:
# in einfacheren Worten erklären, welcher Wert gelöscht werden muss
def get_value_to_be_deleted(question):
    if question == question_list[0]:
        return 'Punktzahl'
    elif question == question_list[1]:
        return 'Klasse'
    elif question == question_list[2]:
        return 'Aufgabe'
    elif (question == question_list[3]) or (question == question_list[4]):
        return 'Zeitpunkt'
    elif question == question_list[5]:
        return 'Fach'      
    else:
        return 'Name'

In [ ]:
def delete_data(statement, value, verb):
    if statement['verb']['id'] == 'https://adlnet.gov/expapi/initialized':
        delete_data_initialized(statement, value)
    elif statement['verb']['id'] == 'http://id.tincanapi.com/verb/viewed':
        delete_data_viewed(statement, value)
    elif statement['verb']['id'] == 'https://brindlewaye.com/xAPITerms/verbs/loggedin':
        delete_data_loggedin(statement, value)
    elif statement['verb']['id'] == 'http://adlnet.gov/expapi/verbs/answered':
        delete_data_answered(statement, value)
    elif statement['verb']['id'] == 'http://activitystrea.ms/schema/1.0/receive':
        delete_data_receive(statement, value)
    elif statement['verb']['id'] == 'https://brindlewaye.com/xAPITerms/verbs/loggedout':
        delete_data_loggedout(statement, value)
    elif statement['verb']['id'] == 'http://adlnet.gov/expapi/verbs/completed':
        delete_data_completed(statement, value)
    elif statement['verb']['id'] == 'http://activitystrea.ms/schema/1.0/start':
        delete_data_start(statement, value)
    elif statement['verb']['id'] == 'https://adlnet.gov/expapi/launched':
        delete_data_launched(statement, value)
    elif statement['verb']['id'] == 'http://adlnet.gov/expapi/verbs/registered':
        delete_data_registered(statement, value)
    else:
        delete_data_terminated(statement, value)

In [ ]:
# Ersetzt die zu löschenden Daten durch 'deleted' für bessere Lesbarkeit.
# Um Attribute zu löschen:  
# statement['actor']['name'] = 'deleted'
# durch
# del statement['actor']['name']
# ersetzen

def delete_data_initialized(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        pass
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
        statement['object']['definition']['description'][language] = 'deleted'
        statement['context']['contextActivities']['parent'][0]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['parent'][0]['definition']['description'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_viewed(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['object']['definition']['name'][language] = 'deleted'
        statement['object']['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_loggedin(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['object']['definition']['name'][language] = 'deleted'
    elif value == 'Aufgabe':
        pass
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_answered(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
    elif value == 'Aufgabe':
        statement['context']['contextActivities']['grouping'][2]['definition']['name'][language] = 'deleted'
        # aufgabe ebenfalls löschen
        # statement['object']['defnition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        # response und success löschen
        # statement['result']['response'] = 'deleted'
        # statement['result']['success'] = 'deleted'
        pass
    else:
        pass

def delete_data_receive(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
        statement['context']['instructor']['name'] = 'deleted'
        statement['context']['instructor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['grouping'][2]['definition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_loggedout(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        pass
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_completed(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        if 'result' in statement:
            statement['result']['score']['raw'] = 'deleted'
            statement['result']['score']['scaled'] = 'deleted'
        # success löschen
        # statement['result']['success'] = 'deleted'
    else:
        pass

def delete_data_start(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_launched(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        pass
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
        statement['object']['definition']['description'][language] = 'deleted'
        statement['context']['contextActivities']['parent'][0]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['parent'][0]['definition']['description'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_registered(statement, value):
    language = statement['context']['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['object']['definition']['name'][language] = 'deleted'
        statement['object']['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
    elif value == 'Aufgabe':
        pass
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

def delete_data_terminated(statement, value):
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
    elif value == 'Klasse':
        pass
    elif value == 'Aufgabe':
        statement['object']['definition']['name'] = {}
        statement['object']['definition']['description'] = {}
        statement['context']['contextActivities']['parent'][0]['definition']['name'][language] = 'deleted'
        statement['context']['contextActivities']['parent'][0]['definition']['description'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        pass
    else:
        pass

In [ ]:
def delete_statement_data(statement_ids, question, start_time, end_time):
    for statement_id in statement_ids:
        i = 0
        while i < len(updated_data):
            if ((statement_id == updated_data[i]['id']) & (updated_data[i]['timestamp'] > start_time)):
                verb = updated_data[i]['verb']['id']
                # "no" end time == current answer
                if end_time == 0:
                    value = get_value_to_be_deleted(question)
                    delete_data(updated_data[i], value, verb)
                    print(statement_id)
                else:
                    value = get_value_to_be_deleted(question)
                    delete_data(updated_data[i], value, verb)
                    print(statement_id)
            i += 1

In [ ]:
def get_statement_ids(time1, time2, name):
    i = 0
    statement_ids = []
    while i < len(updated_data):
        try:
            if time2 == 0:
                if (updated_data[i]['timestamp'] > time1) & (updated_data[i]['actor']['name'] == name):
                    statement_ids.append(updated_data[i]['id'])  
            else:
                if (updated_data[i]['timestamp'] > time1) & (updated_data[i]['timestamp'] < time2) & (updated_data[i]['actor']['name'] == name):
                    statement_ids.append(updated_data[i]['id'])  
        except KeyError as e:
            statement_ids = statement_ids
        i += 1
    return statement_ids

In [ ]:
# diese Funktion überprüft und entfernt die Daten
# gibt Statement IDs aus, wenn das Statement geändert wurde

for name in name_list:
    i = 0
    times = df['Zeitpunkt'].unique().tolist()
    if len(times) >= 1:
        t = 0
        for time in times:
            for question in question_list:
                if(t == len(times)-1):
                    break
                j = 0
                while j < 7:
                    if question == df.at[j, 'Frage'] and df.at[j, 'Antwort'] == 'Falsch':
                        statement_ids = get_statement_ids(times[t], times[t+1], name)
                        delete_statement_data(statement_ids, question, times[t], times[t+1])
                    j += 1
                
                i += 1
            t += 1
        
        for question in question_list:
            j = 7*(len(times)-1)
            while j < 7*len(times):
                if question == df.at[j, 'Frage'] and df.at[j, 'Antwort'] == 'Falsch':
                    statement_ids = get_statement_ids(times[-1], 0, name)
                    delete_statement_data(statement_ids, question, times[-1], times[0])
                j += 1
                
    else:
        for time in times:
            for question in question_list:
                j = 0
                while j < 7*len(times):
                    if question == df.at[j, 'Frage'] and df.at[j, 'Antwort'] == 'Falsch':
                        statement_ids = get_statement_ids(times[-1], 0, name)
                        delete_statement_data(statement_ids, question, times[-1], times[0])
                    j += 1

# Statements in neue Datei schreiben

In [ ]:
with open('new_statements.json', 'w') as new_file:
    json.dump(updated_data, new_file)

# Test: Für jedes Verb überprüfen, ob die Struktur gleich ist

<div class="alert alert-block alert-warning">
<b>Nicht umfangreich genug, das Dictionary ist zu sehr verschachtelt.</b>
</div>

In [ ]:
with open('moodle-dashboard.json') as file:
    data = json.load(file)

In [ ]:
# verb = das Verb, dass die gesuchten Statements vereint
def get_statements(verb):
    statements = []
    for s in data:
        if (s['verb']['id'] == 'https://adlnet.gov/expapi/' + verb):
            statements.append(s)
    return statements

# überprüft, ob die übergebenen Dictionaries die gleichen Keys haben. 
# Es reicht, dass das erste Statement mit allen anderen verglichen wird
def check_statements(statements):
    i = 0
    if len(statements) > 1:
        statement = statements[0]
    
    while i < len(statements):
        if statement.keys() != statements[i].keys():
            print('Not the same!')
            print(statements[i])
            break;
        i += 1
    print('Check! Keys are the same!')
        
    

In [ ]:
initialized = get_statements('initialized')
check_statements(initialized)

viewed = get_statements('viewed')
check_statements(viewed)

loggedin = get_statements('loggedin')
check_statements(loggedin)

answered = get_statements('answered')
check_statements(answered)

received = get_statements('receive')
check_statements(received)

loggedout = get_statements('loggedout')
check_statements(loggedout)

completed = get_statements('completed')
check_statements(completed)

start = get_statements('start')
check_statements(start)

launched = get_statements('launched')
check_statements(launched)

registered = get_statements('registered')
check_statements(registered)

terminated = get_statements('terminated')
check_statements(terminated)
